# Tutorial: Thermal Conductivity
In this tutorial we demonstrate how to compute the effective thermal conductivity of a material based on its microstructure and constituent properties. In this example, we compute the thermal conductivity of FiberForm, a carbon fiber based material. 

Note: The sample size used in this example is very small, well below the size need for a representative volume of the sample. 

In [5]:
# ONLY FOR GOOGLE COLAB: Run this cell only the first time you open a tutorial
if 'google.colab' in str(get_ipython()):
    !pip install -q condacolab
    import condacolab
    condacolab.install()
    !conda install -c fsemerar puma

## Isotropic conductivity

First, we must import puma:

In [1]:
import numpy as np
import os
import sys
import pumapy as puma
import pyvista as pv

Next we will show a thermal conductivity simulation based on a non-segmented representation of the material. In the example material used, the void phase is contained in grayscale range [0,89] and the solid phase is contained in the grayscale range of [90,255]. This range varies for each tomography sample.

The outputs of the thermal conductivity simulation are the effective thermal conductivity tensor, the steady state temperature profile, and the steady state heat flux. 

In [7]:
# Import an example tomography file of size 200^3 and voxel length 1.3e-6
ws_fiberform = puma.import_3Dtiff(puma.path_to_example_file("200_fiberform.tif"), 1.3e-6)

# Cropping the 200x200x200 image into a 100x100x100 sample
ws_fiberform.matrix = ws_fiberform.matrix[50:150, 50:150, 50:150]

# Generating a conductivity map. This stores the conductivity values for each phase of the material
cond_map = puma.IsotropicConductivityMap()
# First, we set the conductivity of the void phase to be 0.0257 (air at STP)
cond_map.add_material((0, 89), 0.0257)
# Next we set the conductivity of the solid phase to be 12 (typical value for carbon fiber)
cond_map.add_material((90, 255), 12)

# The thermal conductivity calculation needs to be run for each of the three simulation directions. 
# For each simulation, a temperature gradient is forced in the simulation direction, and converged to steady state

# Simulation inputs: 
#.  1. workspace - the computational domain for the simulation, containing your material microstructure
#.  2. cond_map - the conductivity values for each material phase
#.  3. direction - the simulation direction, 'x', 'y', or 'z'
#.  4. side_bc - boundary condition in the non-simulation direction. Can be 'p' - periodic, 's' - symmetric, 'd' - dirichlet
#.  5. tolerance - accuracy of the numerical solver, defaults to 1e-4. 
#.  6. maxiter - maximum number of iterations, defaults to 10,000
#.  7. solver_type - the iterative solver used. Can be 'bicgstab', 'cg', 'gmres', or 'direct'. Defaults to 'bicgstab'

k_eff_x, T_x, q_x = puma.compute_thermal_conductivity(ws_fiberform, cond_map, 'x', 's', tolerance=1e-4, solver_type='cg')

print("Effective thermal conductivity tensor:")
print(k_eff_x)

Importing /Users/fsemerar/Documents/PuMA_playground/puma-dev/python/pumapy/data/200_fiberform.tif ... Done
Creating conductivity matrix ... Done
Initializing temperature field ... Done
Setting up b matrix ... Done
Assembling A matrix ... 100.0% Done
Setting up preconditioner ...Done
Time to sep up A matrix:  0.47483777499974167
Solving Ax=b system ... Conjugate Gradient:
Iteration 1301  Residual = 0.0009861697937639732  ... Done
Time to solve:  28.17984823100005
Computing flux from converged temperature field ... Done
Computing effective conductivity... Time to compute fluxes:  0.04280447400014964
Creating conductivity matrix ... Done
Initializing temperature field ... Done
Setting up b matrix ... Done
Assembling A matrix ... 100.0% Done
Setting up preconditioner ...Done
Time to sep up A matrix:  0.31928571500020553
Solving Ax=b system ... Conjugate Gradient:
Iteration 689  Residual = 0.0009856475778077375  ... Done
Time to solve:  15.175427306000074
Computing flux from converged tempe

In [8]:
# Visualizing one slice of the temperature field: 
puma.render_volume(T_x, solid_color=None, notebook=True, cmap='jet')

ViewInteractiveWidget(height=1200, layout=Layout(height='auto', width='100%'), width=1920)

In [20]:
# Visualizing the flux magnitude: 
puma.render_volume(np.linalg.norm(q_x, axis=3), notebook=True, cmap='jet')

ViewInteractiveWidget(height=1200, layout=Layout(height='auto', width='100%'), width=1920)

Below is an example of a thermal conductivity simulation along the y axis, but now performed on a segmented image in order to show an alternative option.

In [9]:
# Segments the image. All values >= 90 are set to 1, and all values <90 are set to 0.
ws_fiberform.binarize(90)

# Generating a conductivity map. This stores the conductivity values for each phase of the material
cond_map = puma.IsotropicConductivityMap()
# First, we set the conductivity of the void phase to be 0.0257 (air at STP)
cond_map.add_material((0, 0), 0.0257)
# Next we set the conductivity of the solid phase to be 12 (typical value for carbon fiber)
cond_map.add_material((1, 1), 12)
# Note that the grayscale ranges have changed relative to the last example, since this image is already segmented

# The thermal conductivity calculation needs to be run for each of the three simulation directions. 
# For each simulation, a temperature gradient is forced in the simulation direction, and converged to steady state

# Simulation inputs: 
#.  1. workspace - the computational domain for the simulation, containing your material microstructure
#.  2. cond_map - the conductivity values for each material phase
#.  3. direction - the simulation direction, 'x', 'y', or 'z'
#.  4. side_bc - boundary condition in the non-simulation direction. Can be 'p' - periodic, 's' - symmetric, 'd' - dirichlet
#.  5. tolerance - accuracy of the numerical solver, defaults to 1e-4. 
#.  6. maxiter - maximum number of iterations, defaults to 10,000
#.  7. solver_type - the iterative solver used. Can be 'bicgstab', 'cg', 'gmres', or 'direct'. Defaults to 'bicgstab'

k_eff_y, T_y, q_y = puma.compute_thermal_conductivity(ws_fiberform, cond_map, 'y', 's', tolerance=1e-4, solver_type='cg')

print("Effective thermal conductivity tensor:")
print(k_eff_y)

Importing /Users/fsemerar/Documents/PuMA_playground/puma-dev/python/pumapy/data/200_fiberform.tif ... Done
Creating conductivity matrix ... Done
Initializing temperature field ... Done
Setting up b matrix ... Done
Assembling A matrix ... 100.0% Done
Setting up preconditioner ...Done
Time to sep up A matrix:  0.3681787200002873
Solving Ax=b system ... Conjugate Gradient:
Iteration 1301  Residual = 0.0009861697937639732  ... Done
Time to solve:  28.10230088399976
Computing flux from converged temperature field ... Done
Computing effective conductivity... Time to compute fluxes:  0.04330263600013495
Creating conductivity matrix ... Done
Initializing temperature field ... Done
Setting up b matrix ... Done
Assembling A matrix ... 100.0% Done
Setting up preconditioner ...Done
Time to sep up A matrix:  0.3424343599999702
Solving Ax=b system ... Conjugate Gradient:
Iteration 689  Residual = 0.0009856475778077375  ... Done
Time to solve:  15.856293318000098
Computing flux from converged tempera

In [17]:
# Visualizing the temperature field: 
puma.render_volume(T_y, solid_color=None, notebook=True, cmap='jet')

ViewInteractiveWidget(height=1200, layout=Layout(height='auto', width='100%'), width=1920)

In [18]:
# Visualizing the flux magnitude: 
puma.render_volume(np.linalg.norm(q_y, axis=3), notebook=True, cmap='jet')

ViewInteractiveWidget(height=1200, layout=Layout(height='auto', width='100%'), width=1920)

## Anisotropic conductivity
Next we show how to compute the conductivity is the constituent phases are anisotropic themselves. This solver is significantly slower because of the higher complexity of the numerical scheme used. For this reason, we scale the domain by half in order to keep the runtime reasonable.

In [ ]:
# Import an example tomography file of size 100^3 and voxel length 1.3e-6
ws_fiberform = puma.import_3Dtiff(puma.path_to_example_file("200_fiberform.tif"), 1.3e-6)
ws_fiberform.rescale(0.5, False)

# detect the fiber orientation using the Structure Tensor
puma.compute_orientation_st(ws_fiberform, sigma=1.4, rho=0.7, cutoff=(90, 255))

In [4]:
# visualize the detected orientation field
puma.render_orientation(ws_fiberform, notebook=True)

ViewInteractiveWidget(height=1200, layout=Layout(height='auto', width='100%'), width=1920)

In [3]:
# Generating a conductivity map. This stores the conductivity values for each phase of the material
cond_map = puma.AnisotropicConductivityMap()
# First, we set the conductivity of the void phase to be 0.0257 (air at STP)
cond_map.add_isotropic_material((0, 89), 0.0257)
# Next we set the conductivity of the fiber phase to be 12 along the fiber and 0.7 across it
cond_map.add_material_to_orient((90, 255), 12., 0.7)

# Simulation inputs: 
#.  1. workspace - the computational domain for the simulation, containing your material microstructure
#.  2. cond_map - the conductivity values for each material phase
#.  3. direction - the simulation direction, 'x', 'y', or 'z'
#.  4. side_bc - boundary condition in the non-simulation direction. Can be 'p' - periodic, 's' - symmetric, 'd' - dirichlet
#.  5. tolerance - accuracy of the numerical solver, defaults to 1e-4. 
#.  6. maxiter - maximum number of iterations, defaults to 10,000
#.  7. solver_type - the iterative solver used. Can be 'bicgstab', 'cg', 'gmres', or 'direct'. Defaults to 'bicgstab'

# When an anisotropic conductivity is fed, the solver automatically uses the MPFA finite volume method
k_eff_z, T_z, q_z = puma.compute_thermal_conductivity(ws_fiberform, cond_map, 'z', 's')

print("Effective thermal conductivity tensor:")
print(k_eff_z)

Importing /Users/fsemerar/Documents/PuMA_playground/puma-dev/python/pumapy/data/200_fiberform.tif ... Done
Rescaled workspace size: (100, 100, 100)
First gradient computation ... Done
Blurring of gradients ... Done
Computing eigenvalue analysis ... Done
Initializing and padding domains ... Done
Assembling b vector ... Done
Initializing large data structures ... Done
Assembling A matrix ... 100.0% Done
Solving Ax=b system ... Bicgstab:
Iteration 566  Residual = 0.0009831141475288565  ... Done
Computing fluxes ... 100.0% Done
Effective thermal conductivity tensor:
[-0.00800957052572341, 0.03337999395366265, 0.08958068952447776]
